In [1]:
import numpy as np
import pandas as pd
from os import path
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.callbacks import EarlyStopping
from collections import Counter
import sklearn
from tensorflow.keras.callbacks import LearningRateScheduler

from sklearn import preprocessing, metrics
from sklearn.metrics import (roc_curve, auc, accuracy_score, precision_score, 
                             recall_score, f1_score, balanced_accuracy_score, 
                             matthews_corrcoef)
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, LabelBinarizer
import shap
from imblearn.over_sampling import RandomOverSampler
import innvestigate

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)
np.random.seed(0)

tf.compat.v1.disable_eager_execution()

2024-06-11 18:06:20.142613: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/home/oarreche@ads.iu.edu/anaconda3/envs/tf23/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### DNN Regression
It is now time to implements single-input and multiple-inputs DNN models

In [2]:
output_file_name = "NSL_Base.txt"
with open(output_file_name, "w") as f: print('---------------------------------------------------------------------------------', file = f)


In [3]:

def confusion_metrics (name_model,predictions,true_labels,time_taken):

    name = name_model
    pred_label = predictions
    y_test_01 = true_labels 

    with open(output_file_name, "a") as f: print('--------------------------------------------------------------------------', file = f)

    with open(output_file_name, "a") as f: print(name, file = f)


    print('---------------------------------------------------------------------------------')
    print('CONFUSION MATRIX')
    print('---------------------------------------------------------------------------------')


    confusion_matrix = pd.crosstab(y_test_01, pred_label,rownames=['Actual ALERT'],colnames = ['Predicted ALERT'], dropna=False).sort_index(axis=0).sort_index(axis=1)
    all_unique_values = sorted(set(pred_label) | set(y_test_01))
    z = np.zeros((len(all_unique_values), len(all_unique_values)))
    rows, cols = confusion_matrix.shape
    z[:rows, :cols] = confusion_matrix
    confusion_matrix  = pd.DataFrame(z, columns=all_unique_values, index=all_unique_values)
    # confusion_matrix.to_csv('Ensemble_conf_matrix.csv')
    # with open(output_file_name, "a") as f:print(confusion_matrix,file=f)
    print(confusion_matrix)
    with open(output_file_name, "a") as f: print('Confusion Matrix', file = f)

    with open(output_file_name, "a") as f: print(confusion_matrix, file = f)


    FP = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)
    FN = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix)
    TP = np.diag(confusion_matrix)
    TN = confusion_matrix.values.sum() - (FP + FN + TP)
    TP_total = sum(TP)
    TN_total = sum(TN)
    FP_total = sum(FP)
    FN_total = sum(FN)

    TP_total = np.array(TP_total,dtype=np.float64)
    TN_total = np.array(TN_total,dtype=np.float64)
    FP_total = np.array(FP_total,dtype=np.float64)
    FN_total = np.array(FN_total,dtype=np.float64)
    FPR = FP / (FP + TN)
    FPR = 100*(sum(FPR)/len(FPR))

    #----------------------------------------------------------------#----------------------------------------------------------------

    print('---------------------------------------------------------------------------------')
    print('METRICS')
    print('---------------------------------------------------------------------------------')


    Acc = accuracy_score(y_test_01, pred_label)
    Precision = precision_score(y_test_01, pred_label, average='macro')
    Recall = recall_score(y_test_01, pred_label, average='macro')
    F1 =  f1_score(y_test_01, pred_label, average='macro')
    BACC = balanced_accuracy_score(y_test_01, pred_label)
    MCC = matthews_corrcoef(y_test_01, pred_label)

    print('Accuracy total: ', Acc)
    print('Precision total: ', Precision )
    print('Recall total: ', Recall )
    print('F1 total: ', F1 )
    print('BACC total: ', BACC)
    print('MCC total: ', MCC)

    with open(output_file_name, "a") as f: print('Accuracy total: ', Acc, file = f)
    with open(output_file_name, "a") as f: print('Precision total: ', Precision, file = f)
    with open(output_file_name, "a") as f: print('Recall total: ', Recall , file = f)
    with open(output_file_name, "a") as f: print('F1 total: ', F1, file = f)
    with open(output_file_name, "a") as f: print('BACC total: ', BACC , file = f)
    with open(output_file_name, "a") as f: print('MCC total: ', MCC, file = f)
    with open(output_file_name, "a") as f: print('Time Taken: ', time_taken, file = f)
    with open(output_file_name, "a") as f: print('FPR: ', FPR, '%' ,file = f)

    return Acc, Precision, Recall, F1, BACC, MCC, FPR



# attach the column names to the dataset
feature=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot",
          "num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
          "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate",
          "rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count", 
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
          "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"]
# KDDTrain+_2.csv & KDDTest+_2.csv are the datafiles without the last column about the difficulty score
# these have already been removed.

train='KDDTrain+.txt'
test='KDDTest+.txt'

df=pd.read_csv(train,names=feature)
df_test=pd.read_csv(test,names=feature)



# shape, this gives the dimensions of the dataset
print('Dimensions of the Training set:',df.shape)
print('Dimensions of the Test set:',df_test.shape)


df.drop(['difficulty'],axis=1,inplace=True)
df_test.drop(['difficulty'],axis=1,inplace=True)



print('Label distribution Training set:')
print(df['label'].value_counts())
print()
print('Label distribution Test set:')
print(df_test['label'].value_counts())



# colums that are categorical and not binary yet: protocol_type (column 2), service (column 3), flag (column 4).
# explore categorical features
print('Training set:')
for col_name in df.columns:
    if df[col_name].dtypes == 'object' :
        unique_cat = len(df[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))

#see how distributed the feature service is, it is evenly distributed and therefore we need to make dummies for all.
print()
print('Distribution of categories in service:')
print(df['service'].value_counts().sort_values(ascending=False).head())



# Test set
print('Test set:')
for col_name in df_test.columns:
    if df_test[col_name].dtypes == 'object' :
        unique_cat = len(df_test[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} categories".format(col_name=col_name, unique_cat=unique_cat))


from sklearn.preprocessing import LabelEncoder,OneHotEncoder
categorical_columns=['protocol_type', 'service', 'flag']
# insert code to get a list of categorical columns into a variable, categorical_columns
categorical_columns=['protocol_type', 'service', 'flag'] 
 # Get the categorical values into a 2D numpy array
df_categorical_values = df[categorical_columns]
testdf_categorical_values = df_test[categorical_columns]
df_categorical_values.head()


# protocol type
unique_protocol=sorted(df.protocol_type.unique())
string1 = 'Protocol_type_'
unique_protocol2=[string1 + x for x in unique_protocol]
# service
unique_service=sorted(df.service.unique())
string2 = 'service_'
unique_service2=[string2 + x for x in unique_service]
# flag
unique_flag=sorted(df.flag.unique())
string3 = 'flag_'
unique_flag2=[string3 + x for x in unique_flag]
# put together
dumcols=unique_protocol2 + unique_service2 + unique_flag2
print(dumcols)

#do same for test set
unique_service_test=sorted(df_test.service.unique())
unique_service2_test=[string2 + x for x in unique_service_test]
testdumcols=unique_protocol2 + unique_service2_test + unique_flag2


df_categorical_values_enc=df_categorical_values.apply(LabelEncoder().fit_transform)
print(df_categorical_values_enc.head())
# test set
testdf_categorical_values_enc=testdf_categorical_values.apply(LabelEncoder().fit_transform)



enc = OneHotEncoder()
df_categorical_values_encenc = enc.fit_transform(df_categorical_values_enc)
df_cat_data = pd.DataFrame(df_categorical_values_encenc.toarray(),columns=dumcols)
# test set
testdf_categorical_values_encenc = enc.fit_transform(testdf_categorical_values_enc)
testdf_cat_data = pd.DataFrame(testdf_categorical_values_encenc.toarray(),columns=testdumcols)

df_cat_data.head()


trainservice=df['service'].tolist()
testservice= df_test['service'].tolist()
difference=list(set(trainservice) - set(testservice))
string = 'service_'
difference=[string + x for x in difference]
difference

for col in difference:
    testdf_cat_data[col] = 0

testdf_cat_data.shape

newdf=df.join(df_cat_data)
newdf.drop('flag', axis=1, inplace=True)
newdf.drop('protocol_type', axis=1, inplace=True)
newdf.drop('service', axis=1, inplace=True)
# test data
newdf_test=df_test.join(testdf_cat_data)
newdf_test.drop('flag', axis=1, inplace=True)
newdf_test.drop('protocol_type', axis=1, inplace=True)
newdf_test.drop('service', axis=1, inplace=True)
print(newdf.shape)
print(newdf_test.shape)


# take label column
labeldf=newdf['label']
labeldf_test=newdf_test['label']
# change the label column
newlabeldf=labeldf.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
newlabeldf_test=labeldf_test.replace({ 'normal' : 0, 'neptune' : 1 ,'back': 1, 'land': 1, 'pod': 1, 'smurf': 1, 'teardrop': 1,'mailbomb': 1, 'apache2': 1, 'processtable': 1, 'udpstorm': 1, 'worm': 1,
                           'ipsweep' : 2,'nmap' : 2,'portsweep' : 2,'satan' : 2,'mscan' : 2,'saint' : 2
                           ,'ftp_write': 3,'guess_passwd': 3,'imap': 3,'multihop': 3,'phf': 3,'spy': 3,'warezclient': 3,'warezmaster': 3,'sendmail': 3,'named': 3,'snmpgetattack': 3,'snmpguess': 3,'xlock': 3,'xsnoop': 3,'httptunnel': 3,
                           'buffer_overflow': 4,'loadmodule': 4,'perl': 4,'rootkit': 4,'ps': 4,'sqlattack': 4,'xterm': 4})
# put the new label column back
newdf['label'] = newlabeldf
newdf_test['label'] = newlabeldf_test
print(newdf['label'].head())


# Specify your selected features. Note that you'll need to modify this list according to your final processed dataframe
#Uncomment the below lines to use these top 20 features from shap analysis
#selected_features = ["root_shell","service_telnet","num_shells","service_uucp","dst_host_same_src_port_rate"
#                     ,"dst_host_rerror_rate","dst_host_srv_serror_rate","dst_host_srv_count","service_private","logged_in",
#                    "dst_host_serror_rate","serror_rate","srv_serror_rate","flag_S0","diff_srv_rate","dst_host_srv_diff_host_rate","num_file_creations","flag_RSTR"#,"dst_host_same_srv_rate","service_Idap","label"]
                     

# Select those features from your dataframe
#newdf = newdf[selected_features]
#newdf_test = newdf_test[selected_features]

# Now your dataframe only contains your selected features.

# creating a dataframe with multi-class labels (Dos,Probe,R2L,U2R,normal)
multi_data = newdf.copy()
multi_label = pd.DataFrame(multi_data.label)

multi_data_test=newdf_test.copy()
multi_label_test = pd.DataFrame(multi_data_test.label)


# using standard scaler for normalizing
std_scaler = StandardScaler()
def standardization(df,col):
    for i in col:
        arr = df[i]
        arr = np.array(arr)
        df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    return df

numeric_col = multi_data.select_dtypes(include='number').columns
data = standardization(multi_data,numeric_col)
numeric_col_test = multi_data_test.select_dtypes(include='number').columns
data_test = standardization(multi_data_test,numeric_col_test)

# label encoding (0,1,2,3,4) multi-class labels (Dos,normal,Probe,R2L,U2R)
le2 = preprocessing.LabelEncoder()
le2_test = preprocessing.LabelEncoder()
enc_label = multi_label.apply(le2.fit_transform)
enc_label_test = multi_label_test.apply(le2_test.fit_transform)
multi_data = multi_data.copy()
multi_data_test = multi_data_test.copy()

multi_data['intrusion'] = enc_label
multi_data_test['intrusion'] = enc_label_test

#y_mul = multi_data['intrusion']
multi_data
multi_data_test



multi_data.drop(labels= [ 'label'], axis=1, inplace=True)
multi_data
multi_data_test.drop(labels= [ 'label'], axis=1, inplace=True)
multi_data_test


y_train_multi= multi_data[['intrusion']]
X_train_multi= multi_data.drop(labels=['intrusion'], axis=1)

print('X_train has shape:',X_train_multi.shape,'\ny_train has shape:',y_train_multi.shape)

y_test_multi= multi_data_test[['intrusion']]
X_test_multi= multi_data_test.drop(labels=['intrusion'], axis=1)

print('X_test has shape:',X_test_multi.shape,'\ny_test has shape:',y_test_multi.shape)


from collections import Counter

label_counts = Counter(y_train_multi['intrusion'])
print(label_counts)


from sklearn.preprocessing import LabelBinarizer

y_train_multi = LabelBinarizer().fit_transform(y_train_multi)

y_test_multi = LabelBinarizer().fit_transform(y_test_multi)


Y_train=y_train_multi.copy()
X_train=X_train_multi.copy()

Y_test=y_test_multi.copy()
X_test=X_test_multi.copy()


from imblearn.over_sampling import RandomOverSampler
from sklearn.datasets import make_classification

# Assuming you have features X and labels Y
# X, Y = make_classification()

ros = RandomOverSampler(sampling_strategy='minority', random_state=100)

X_train, Y_train = ros.fit_resample(X_train, Y_train)


single_class_train = np.argmax(y_train_multi, axis=1)
single_class_test = np.argmax(y_test_multi, axis=1)


df1 = X_train_multi.assign(Label = single_class_train)
df2 =  X_test_multi.assign(Label = single_class_test)

frames = [df1,  df2]

df = pd.concat(frames,ignore_index=True)

# IG popping the features

# df.pop('same_srv_rate')
# df.pop('service_http')
# df.pop('flag_SF')
# df.pop('dst_host_count')
# df.pop('service_private')

# df.pop('logged_in')
# df.pop('count')
# df.pop('dst_host_serror_rate')
# df.pop('flag_S0')
# df.pop('Protocol_type_tcp')

# df.pop('dst_host_srv_serror_rate')
# df.pop('srv_diff_host_rate')
# df.pop('dst_host_same_srv_rate')
# df.pop('srv_serror_rate')
# df.pop('dst_host_srv_count')
# df.pop('serror_rate')
# df.pop('dst_host_same_src_port_rate')
# df.pop('Protocol_type_udp')
# df.pop('service_ecr_i')
# df.pop('srv_count')
# df.pop('srv_rerror_rate')

# df.pop('dst_host_srv_rerror_rate')
# df.pop('rerror_rate')
# df.pop('dst_host_srv_diff_host_rate')
# df.pop('diff_srv_rate')
# df.pop('service_other')
# df.pop('service_ftp_data')
# df.pop('service_smtp')
# df.pop('dst_host_rerror_rate')
# df.pop('dst_host_diff_srv_rate')
# df.pop('Protocol_type_icmp')
# df.pop('flag_REJ')
# df.pop('service_telnet')
# df.pop('hot')
# df.pop('service_eco_i')
# df.pop('flag_RSTR')
# df.pop('service_domain_u')
# df.pop('service_ftp')
# df.pop('flag_RSTO')
# df.pop('is_guest_login')
# df.pop('service_Z39_50')
# df.pop('duration')

# df.pop('service_iso_tsap')
# df.pop('service_pop_3')
# df.pop('service_ldap')
# df.pop('service_efs')
# df.pop('service_http_443')
# df.pop('service_daytime')
# df.pop('service_finger')
# df.pop('service_sunrpc')
# df.pop('service_systat')
# df.pop('service_echo')
# df.pop('service_discard')
# df.pop('service_uucp')
# df.pop('service_uucp_path')
# df.pop('service_mtp')
# df.pop('service_urp_i')
# df.pop('service_vmnet')
# df.pop('service_netbios_dgm')
# df.pop('service_courier')
# df.pop('service_name')
# df.pop('service_whois')
# df.pop('service_imap4')
# df.pop('service_csnet_ns')
# df.pop('service_login')
# df.pop('service_netstat')
# df.pop('service_kshell')
# df.pop('service_klogin')
# df.pop('service_netbios_ns')
# df.pop('service_bgp')
# df.pop('service_exec')
# df.pop('service_nnsp')
# df.pop('service_hostnames')
# df.pop('service_ctf')
# df.pop('service_link')
# df.pop('service_ssh')
# df.pop('wrong_fragment')
# df.pop('service_gopher')
# df.pop('service_sql_net')


# df.pop('service_netbios_ssn')
# df.pop('service_time')
# df.pop('flag_S1')
# df.pop('service_supdup')
# df.pop('flag_SH')
# df.pop('num_failed_logins')
# df.pop('service_domain')
# df.pop('root_shell')
# df.pop('dst_bytes')
# df.pop('num_access_files')
# df.pop('service_remote_job')
# df.pop('flag_RSTOS0')
# df.pop('service_auth')
# df.pop('service_shell')
# df.pop('service_pop_2')
# df.pop('service_IRC')
# df.pop('service_ntp_u')
# df.pop('flag_S3')
# df.pop('service_nntp')
# df.pop('flag_S2')
# df.pop('service_rje')
# df.pop('num_shells')
# df.pop('su_attempted')
# df.pop('service_printer')
# df.pop('num_file_creations')
# df.pop('service_X11')
# df.pop('src_bytes')
# df.pop('land')
# df.pop('flag_OTH')
# df.pop('num_root')
# df.pop('num_compromised')
# df.pop('service_tim_i')
# df.pop('service_pm_dump')
# df.pop('service_red_i')
# df.pop('urgent')
# df.pop('service_http_2784')
# df.pop('service_tftp_u')
# df.pop('is_host_login')
# df.pop('service_urh_i')
# df.pop('service_http_8001')
# df.pop('service_harvest')
# df.pop('service_aol')
# df.pop('num_outbound_cmds')



# LRP popping the features

# df.pop('flag_S0')
# df.pop('flag_SF')
# df.pop('srv_serror_rate')
# df.pop('dst_host_count')
# df.pop('dst_host_srv_count')

# df.pop('dst_host_srv_serror_rate') 
# df.pop('serror_rate')
# df.pop('service_http')
# df.pop('count')
# df.pop('dst_host_rerror_rate')

# df.pop('dst_host_same_srv_rate')
# df.pop('same_srv_rate')
# df.pop('logged_in')
# df.pop('Protocol_type_udp')
# df.pop('dst_host_serror_rate')
# df.pop('service_private')
# df.pop('service_ftp_data')
# df.pop('flag_REJ')
# df.pop('service_telnet')
# df.pop('dst_host_srv_diff_host_rate')
# df.pop('dst_host_srv_rerror_rate')

# df.pop('srv_count')
# df.pop('srv_rerror_rate')
# df.pop('service_eco_i')
# df.pop('dst_host_same_src_port_rate')
# df.pop('rerror_rate')
# df.pop('Protocol_type_icmp')
# df.pop('is_guest_login')
# df.pop('service_ftp')
# df.pop('service_iso_tsap')
# df.pop('service_domain_u')
# df.pop('service_uucp_path')
# df.pop('service_Z39_50')
# df.pop('dst_host_diff_srv_rate')
# df.pop('service_smtp')
# df.pop('diff_srv_rate')
# df.pop('service_vmnet')
# df.pop('service_courier')
# df.pop('Protocol_type_tcp')
# df.pop('service_bgp')
# df.pop('flag_RSTO')
# df.pop('srv_diff_host_rate')

# df.pop('service_other')
# df.pop('wrong_fragment')
# df.pop('duration')
# df.pop('service_ldap')
# df.pop('service_ecr_i')
# df.pop('service_login')
# df.pop('service_nntp')
# df.pop('flag_RSTR')
# df.pop('service_nnsp')
# df.pop('hot')
# df.pop('service_finger')
# df.pop('service_urp_i')
# df.pop('service_netbios_dgm')
# df.pop('service_efs')
# df.pop('service_pop_3')
# df.pop('service_uucp')
# df.pop('service_http_443')
# df.pop('service_auth')
# df.pop('flag_SH')
# df.pop('service_imap4')
# df.pop('service_gopher')
# df.pop('service_csnet_ns')
# df.pop('service_sunrpc')
# df.pop('service_klogin')
# df.pop('flag_S1')
# df.pop('service_IRC')
# df.pop('service_ctf')
# df.pop('service_domain')
# df.pop('service_time')
# df.pop('service_whois')
# df.pop('service_supdup')
# df.pop('service_hostnames')
# df.pop('num_failed_logins')
# df.pop('service_ntp_u')
# df.pop('flag_RSTOS0')
# df.pop('service_X11')
# df.pop('service_link')
# df.pop('service_mtp')
# df.pop('service_netstat')
# df.pop('service_discard')
# df.pop('service_pop_2')

# df.pop('service_echo')
# df.pop('service_exec')
# df.pop('service_kshell')
# df.pop('service_systat')
# df.pop('service_netbios_ns')
# df.pop('flag_S2')
# df.pop('service_netbios_ssn')
# df.pop('su_attempted')
# df.pop('num_access_files')
# df.pop('num_file_creations')
# df.pop('service_name')
# df.pop('service_remote_job')
# df.pop('flag_S3')
# df.pop('service_daytime')
# df.pop('service_rje')
# df.pop('num_shells')
# df.pop('root_shell')
# df.pop('service_shell')
# df.pop('service_ssh')
# df.pop('service_printer')
# df.pop('service_urh_i')
# df.pop('service_red_i')
# df.pop('land')
# df.pop('urgent')
# df.pop('service_sql_net')
# df.pop('num_root')
# df.pop('service_http_2784')
# df.pop('service_http_8001')
# df.pop('dst_bytes')
# df.pop('num_compromised')
# df.pop('src_bytes')
# df.pop('flag_OTH')
# df.pop('service_pm_dump')
# df.pop('service_tim_i')
# df.pop('service_tftp_u')
# df.pop('is_host_login')
# df.pop('service_harvest')
# df.pop('service_aol')
# df.pop('num_outbound_cmds')



# SHAP popping

# df.pop('serror_rate')
# df.pop('flag_S0')
# df.pop('dst_host_srv_serror_rate')
# df.pop('flag_SF')
# df.pop('srv_serror_rate')

# df.pop('dst_host_serror_rate')
# df.pop('dst_host_srv_count')
# df.pop('dst_host_same_srv_rate')
# df.pop('service_http')
# df.pop('dst_host_count')

# df.pop('same_srv_rate')
# df.pop('Protocol_type_udp')
# df.pop('dst_host_rerror_rate')
# df.pop('count')
# df.pop('service_ftp_data')
# df.pop('logged_in')
# df.pop('service_private')
# df.pop('dst_host_same_src_port_rate')
# df.pop('service_eco_i')
# df.pop('is_guest_login')

# df.pop('service_telnet')
# df.pop('dst_host_srv_diff_host_rate')
# df.pop('service_Z39_50')
# df.pop('service_iso_tsap')
# df.pop('service_ftp')
# df.pop('flag_REJ')
# df.pop('dst_host_diff_srv_rate')
# df.pop('srv_count')
# df.pop('dst_host_srv_rerror_rate')
# df.pop('diff_srv_rate')
# df.pop('service_pop_3')
# df.pop('service_domain_u')
# df.pop('Protocol_type_icmp')
# df.pop('service_other')
# df.pop('service_bgp')
# df.pop('service_nntp')
# df.pop('service_uucp_path')
# df.pop('service_courier')
# df.pop('rerror_rate')
# df.pop('Protocol_type_tcp')
# df.pop('flag_RSTR')

# df.pop('srv_diff_host_rate')
# df.pop('duration')
# df.pop('srv_rerror_rate')
# df.pop('service_ecr_i')
# df.pop('flag_RSTO')
# df.pop('service_vmnet')
# df.pop('service_smtp')
# df.pop('service_ldap')
# df.pop('service_urh_i')
# df.pop('service_login')
# df.pop('wrong_fragment')
# df.pop('hot')
# df.pop('service_finger')
# df.pop('service_netbios_dgm')
# df.pop('service_IRC')
# df.pop('flag_SH')
# df.pop('service_auth')
# df.pop('service_nnsp')
# df.pop('service_imap4')
# df.pop('service_klogin')
# df.pop('service_discard')
# df.pop('service_time')
# df.pop('service_urp_i')
# df.pop('service_sunrpc')
# df.pop('service_uucp')
# df.pop('service_echo')
# df.pop('land')
# df.pop('service_daytime')
# df.pop('service_mtp')
# df.pop('service_ctf')
# df.pop('service_sql_net')
# df.pop('service_gopher')
# df.pop('service_ntp_u')
# df.pop('service_supdup')
# df.pop('service_csnet_ns')
# df.pop('flag_S1')
# df.pop('flag_RSTOS0')
# df.pop('service_pop_2')
# df.pop('service_netbios_ns')
# df.pop('service_link')
# df.pop('service_http_443')
# df.pop('su_attempted')
# df.pop('flag_S2')

# df.pop('service_X11')
# df.pop('service_netstat')
# df.pop('num_failed_logins')
# df.pop('service_whois')
# df.pop('service_efs')
# df.pop('service_hostnames')
# df.pop('service_domain')
# df.pop('service_kshell')
# df.pop('service_red_i')
# df.pop('num_access_files')
# df.pop('flag_OTH')
# df.pop('service_shell')
# df.pop('root_shell')
# df.pop('service_printer')
# df.pop('service_systat')
# df.pop('service_ssh')
# df.pop('service_exec')
# df.pop('service_netbios_ssn')
# df.pop('service_tftp_u')
# df.pop('flag_S3')
# df.pop('service_rje')
# df.pop('service_name')
# df.pop('num_file_creations')
# df.pop('num_shells')
# df.pop('service_remote_job')
# df.pop('num_root')
# df.pop('num_compromised')
# df.pop('service_tim_i')
# df.pop('dst_bytes')
# df.pop('urgent')
# df.pop('src_bytes')
# df.pop('is_host_login')
# df.pop('service_pm_dump')
# df.pop('service_harvest')
# df.pop('service_http_8001')
# df.pop('service_aol')
# df.pop('service_http_2784')
# df.pop('num_outbound_cmds')



df_fs = df

y = df.pop('Label')
X = df
df = X.assign(Label = y)


# y = df.pop('Label')
# X = df

y1, y2 = pd.factorize(y)

y_0 = pd.DataFrame(y1)
y_1 = pd.DataFrame(y1)
y_2 = pd.DataFrame(y1)
y_3 = pd.DataFrame(y1)
y_4 = pd.DataFrame(y1)


# y_0 = y_0.replace(0, 0)
# y_0 = y_0.replace(1, 1)
y_0 = y_0.replace(2, 1)
y_0 = y_0.replace(3, 1)
y_0 = y_0.replace(4, 1)


y_1 = y_1.replace(1, 999)
y_1 = y_1.replace(0, 1)
# y_1 = y_1.replace(1, 0)
y_1 = y_1.replace(2, 1)
y_1 = y_1.replace(3, 1)
y_1 = y_1.replace(4, 1)
y_1 = y_1.replace(999, 1)


y_2 = y_2.replace(0, 1)
y_2 = y_2.replace(1, 1)
y_2 = y_2.replace(2, 0)
y_2 = y_2.replace(3, 1)
y_2 = y_2.replace(4, 1)


y_3 = y_3.replace(0, 1)
# y_3 = y_3.replace(1, 1)
y_3 = y_3.replace(2, 1)
y_3 = y_3.replace(3, 0)
y_3 = y_3.replace(4, 1)


y_4 = y_4.replace(0, 1)
# y_4 = y_4.replace(1, 1)
y_4 = y_4.replace(2, 1)
y_4 = y_4.replace(3, 1)
y_4 = y_4.replace(4, 0)



df = df.assign(Label = y)


Dimensions of the Training set: (125973, 43)
Dimensions of the Test set: (22544, 43)
Label distribution Training set:
normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: label, dtype: int64

Label distribution Test set:
normal             9711
neptune            4657
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               735
processtable        685
smurf               665
back                359
snmpguess  

In [4]:

# Separate Training and Testing db
print('---------------------------------------------------------------------------------')
print('Separating Training and Testing db')
print('---------------------------------------------------------------------------------')
print('')

X_train,X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=0.7,random_state=42)
df = X.assign( Label = y)


---------------------------------------------------------------------------------
Separating Training and Testing db
---------------------------------------------------------------------------------



In [5]:
y_train

145897    1
77854     1
145078    0
118322    0
116812    0
         ..
119879    1
103694    1
131932    0
146867    3
121958    0
Name: Label, Length: 103961, dtype: int64

In [6]:
print('---------------------------------------------------------------------------------')
print('Defining the DNN model')
print('---------------------------------------------------------------------------------')
print('')


nodes = 5
print(nodes)

model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(len(X_train.columns,))))

model.add(tf.keras.layers.Dense(nodes, activation='relu'))
# model.add(tf.keras.layers.Dense(nodes, activation='relu'))

model.add(tf.keras.layers.Dense(5))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()
#---------------------------------------------------------------------

from tensorflow.keras.callbacks import EarlyStopping
start = time.time()

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)

model.fit(X_train, y_train, epochs=1000, batch_size=len(X_train), callbacks=[early_stopping])

end = time.time()
print('---------------------------------------------------------------------------------')
print('ELAPSE TIME TRAINING MODEL: ',(end - start)/60, 'min')
print('---------------------------------------------------------------------------------')
print('')


---------------------------------------------------------------------------------
Defining the DNN model
---------------------------------------------------------------------------------

5


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 5)                 615       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 30        
Total params: 645
Trainable params: 645
Non-trainable params: 0
_________________________________________________________________
Train on 103961 samples


2024-06-11 18:06:23.903670: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2024-06-11 18:06:23.929894: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:67:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.54GiB deviceMemoryBandwidth: 715.34GiB/s
2024-06-11 18:06:23.930064: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:68:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.53GiB deviceMemoryBandwidth: 715.34GiB/s
2024-06-11 18:06:23.930087: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2024-06-11 18:06:23.946440: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2024-06

Epoch 1/1000
103961/103961 [==============================] - 0s 2us/sample - loss: 6.8063 - accuracy: 0.2256
Epoch 2/1000
103961/103961 [==============================] - 0s 1us/sample - loss: 1.7316 - accuracy: 0.7407
Epoch 3/1000
103961/103961 [==============================] - 0s 1us/sample - loss: 1.0363 - accuracy: 0.8428
Epoch 4/1000
103961/103961 [==============================] - 0s 1us/sample - loss: 1.0229 - accuracy: 0.8377
Epoch 5/1000
103961/103961 [==============================] - 0s 1us/sample - loss: 1.0409 - accuracy: 0.8368
Epoch 6/1000
103961/103961 [==============================] - 0s 1us/sample - loss: 0.9650 - accuracy: 0.8435
Epoch 7/1000
103961/103961 [==============================] - 0s 1us/sample - loss: 0.9161 - accuracy: 0.8434
Epoch 8/1000
103961/103961 [==============================] - 0s 1us/sample - loss: 0.9075 - accuracy: 0.8422
Epoch 9/1000
103961/103961 [==============================] - 0s 1us/sample - loss: 0.9006 - accuracy: 0.8443
Epoch 10/1

In [7]:

print('---------------------------------------------------------------------------------')
print('Model Prediction')
print('---------------------------------------------------------------------------------')
print('')
print('---------------------------------------------------------------------------------')
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
print('ELAPSE TIME MODEL PREDICTION: ',(end - start)/60, 'min')
print('---------------------------------------------------------------------------------')
print('')

#print(y_pred)
ynew = np.argmax(y_pred,axis = 1)
#print(ynew)
score = model.evaluate(X_test, y_test,verbose=1)
#print(score)
label = y2
# pred_label = label[ynew]
#print(score)

label_counts = Counter(y_test)
print(label_counts)

label_counts = Counter(ynew)
print(label_counts)

accuracy =accuracy_score(y_test, ynew)*100
print(accuracy)

---------------------------------------------------------------------------------
Model Prediction
---------------------------------------------------------------------------------

---------------------------------------------------------------------------------


`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


ELAPSE TIME MODEL PREDICTION:  0.007005949815114339 min
---------------------------------------------------------------------------------

Counter({0: 23102, 1: 16081, 2: 4144, 3: 1197, 4: 32})
Counter({0: 22799, 1: 15871, 2: 5736, 3: 150})
84.95601041386121


In [8]:
single_sample_df = X_test.iloc[[0]]

## Integrated Gradients

In [9]:
%%time

# Create an analyzer for the model
analyzer = innvestigate.create_analyzer("integrated_gradients", model)

analysis = analyzer.analyze(X_test)
#uncomment for single sample
# analysis = analyzer.analyze(single_sample_df)
# Print or use the analysis results as needed

# Perform LRP analysis on a certain number of samples
# analysis = analyzer.analyze(X_test.sample(2500))

names = X_test.columns

scores = pd.DataFrame(analysis)

scores_abs = scores.abs()

# Calculate the sum of each column
sum_of_columns = scores_abs.sum(axis=0)

names = list(names)

sum_of_columns = list(sum_of_columns)


# Zip the two lists together
combined = list(zip(names, sum_of_columns))

# Sort the combined list in descending order based on the values from sum_of_columns
sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)

# Unzip the sorted_combined list to separate names and sum_of_columns
sorted_names, sorted_sum_of_columns = zip(*sorted_combined)

print(sorted_names)
print(sorted_sum_of_columns)

sorted_names

`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


('flag_SF', 'logged_in', 'dst_host_same_srv_rate', 'same_srv_rate', 'service_private', 'srv_serror_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'service_http', 'flag_S0', 'dst_host_srv_count', 'srv_diff_host_rate', 'count', 'dst_host_srv_rerror_rate', 'rerror_rate', 'dst_host_count', 'serror_rate', 'srv_rerror_rate', 'dst_host_same_src_port_rate', 'Protocol_type_tcp', 'flag_REJ', 'dst_host_rerror_rate', 'Protocol_type_icmp', 'srv_count', 'service_domain_u', 'diff_srv_rate', 'dst_host_diff_srv_rate', 'service_Z39_50', 'service_ftp_data', 'service_eco_i', 'service_smtp', 'service_iso_tsap', 'wrong_fragment', 'flag_RSTR', 'duration', 'service_nnsp', 'Protocol_type_udp', 'service_whois', 'service_courier', 'service_sunrpc', 'service_ecr_i', 'dst_host_srv_diff_host_rate', 'service_vmnet', 'service_time', 'service_name', 'service_uucp_path', 'service_uucp', 'service_telnet', 'service_bgp', 'service_ftp', 'is_guest_login', 'service_link', 'flag_SH', 'service_http_443', 'service_

('flag_SF',
 'logged_in',
 'dst_host_same_srv_rate',
 'same_srv_rate',
 'service_private',
 'srv_serror_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'service_http',
 'flag_S0',
 'dst_host_srv_count',
 'srv_diff_host_rate',
 'count',
 'dst_host_srv_rerror_rate',
 'rerror_rate',
 'dst_host_count',
 'serror_rate',
 'srv_rerror_rate',
 'dst_host_same_src_port_rate',
 'Protocol_type_tcp',
 'flag_REJ',
 'dst_host_rerror_rate',
 'Protocol_type_icmp',
 'srv_count',
 'service_domain_u',
 'diff_srv_rate',
 'dst_host_diff_srv_rate',
 'service_Z39_50',
 'service_ftp_data',
 'service_eco_i',
 'service_smtp',
 'service_iso_tsap',
 'wrong_fragment',
 'flag_RSTR',
 'duration',
 'service_nnsp',
 'Protocol_type_udp',
 'service_whois',
 'service_courier',
 'service_sunrpc',
 'service_ecr_i',
 'dst_host_srv_diff_host_rate',
 'service_vmnet',
 'service_time',
 'service_name',
 'service_uucp_path',
 'service_uucp',
 'service_telnet',
 'service_bgp',
 'service_ftp',
 'is_guest_login',
 'serv

## LRP

In [10]:
%%time

# Create an analyzer for the model
analyzer = innvestigate.create_analyzer("lrp.z", model)

analysis = analyzer.analyze(X_test)
#uncomment for single sample
# analysis = analyzer.analyze(single_sample_df)

# Perform LRP analysis on a certain number of samples
# analysis = analyzer.analyze(X_test.sample(10000))

# Print or use the analysis results as needed
print(analysis)
print(len(X_test))
print(len(X_test.columns))
names = X_test.columns
print(analysis.shape)
print(type(analysis))
scores = pd.DataFrame(analysis)
print(analysis)
scores_abs = scores.abs()

# Calculate the sum of each column
sum_of_columns = scores_abs.sum(axis=0)

names = list(names)

sum_of_columns = list(sum_of_columns)

# Zip the two lists together
combined = list(zip(names, sum_of_columns))

# Sort the combined list in descending order based on the values from sum_of_columns
sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)

# Unzip the sorted_combined list to separate names and sum_of_columns
sorted_names, sorted_sum_of_columns = zip(*sorted_combined)

print(sorted_names)
print(sorted_sum_of_columns)

sorted_names

[[ 0.039 -0.002 -0.006 ...  0.022  1.365  0.028]
 [ 0.04   0.002 -0.008 ...  0.015  0.684  0.035]
 [ 0.056  0.006 -0.009 ...  0.078  0.595  0.043]
 ...
 [-0.174 -0.002  0.001 ... -0.025  1.393 -0.078]
 [ 0.04   0.002 -0.006 ...  0.015  0.684  0.035]
 [ 0.038 -0.002 -0.006 ...  0.022  1.365  0.028]]
44556
122
(44556, 122)
<class 'numpy.ndarray'>
[[ 0.039 -0.002 -0.006 ...  0.022  1.365  0.028]
 [ 0.04   0.002 -0.008 ...  0.015  0.684  0.035]
 [ 0.056  0.006 -0.009 ...  0.078  0.595  0.043]
 ...
 [-0.174 -0.002  0.001 ... -0.025  1.393 -0.078]
 [ 0.04   0.002 -0.006 ...  0.015  0.684  0.035]
 [ 0.038 -0.002 -0.006 ...  0.022  1.365  0.028]]
('flag_SF', 'same_srv_rate', 'logged_in', 'dst_host_same_srv_rate', 'service_private', 'srv_serror_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'service_http', 'flag_S0', 'dst_host_srv_count', 'srv_diff_host_rate', 'rerror_rate', 'count', 'dst_host_srv_rerror_rate', 'dst_host_count', 'serror_rate', 'srv_rerror_rate', 'dst_host_same_src_p

('flag_SF',
 'same_srv_rate',
 'logged_in',
 'dst_host_same_srv_rate',
 'service_private',
 'srv_serror_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'service_http',
 'flag_S0',
 'dst_host_srv_count',
 'srv_diff_host_rate',
 'rerror_rate',
 'count',
 'dst_host_srv_rerror_rate',
 'dst_host_count',
 'serror_rate',
 'srv_rerror_rate',
 'dst_host_same_src_port_rate',
 'Protocol_type_tcp',
 'flag_REJ',
 'service_domain_u',
 'dst_host_rerror_rate',
 'Protocol_type_icmp',
 'srv_count',
 'diff_srv_rate',
 'dst_host_diff_srv_rate',
 'service_Z39_50',
 'service_eco_i',
 'wrong_fragment',
 'service_iso_tsap',
 'flag_RSTR',
 'service_ftp_data',
 'duration',
 'service_smtp',
 'Protocol_type_udp',
 'service_nnsp',
 'service_whois',
 'service_sunrpc',
 'service_courier',
 'dst_host_srv_diff_host_rate',
 'service_ecr_i',
 'service_time',
 'service_name',
 'service_uucp_path',
 'service_uucp',
 'service_vmnet',
 'service_ftp',
 'is_guest_login',
 'service_bgp',
 'service_telnet',
 'serv

## DeepLift

In [11]:
%%time

with open(output_file_name, "a") as f:print('',file = f)

###here

print('---------------------------------------------------------------------------------')
print('Generating Explainer')
print('---------------------------------------------------------------------------------')

samples = 2500
# uncomment for single sample
# samples = 1

test = X_test
train = X_train
start_index = 0

end_index = samples
explainer = shap.DeepExplainer(model,train[start_index:end_index].values.astype('float'))
shap_values = explainer.shap_values(test[start_index:end_index].values.astype('float'))


vals= np.abs(shap_values).mean(1)
feature_importance = pd.DataFrame(list(zip(train.columns, sum(vals))), columns=['col_name','feature_importance_vals'])
feature_importance.sort_values(by=['feature_importance_vals'], ascending=False,inplace=True)
feature_importance.head()
print(feature_importance.to_string())



print('---------------------------------------------------------------------------------')
feature_val = feature_importance['feature_importance_vals'].tolist()

feature_name = feature_importance['col_name'].tolist()



# Use zip to combine the two lists, sort based on list1, and then unzip them
zipped_lists = list(zip(feature_name, feature_val))
zipped_lists.sort(key=lambda x: x[1],reverse=True)

# Convert the sorted result back into separate lists
sorted_list1, sorted_list2 = [list(x) for x in zip(*zipped_lists)]

for k in sorted_list1:
  with open(output_file_name, "a") as f:print("df.pop('",k,"')", sep='',file = f)
with open(output_file_name, "a") as f:print("Trial_ =[", file = f)
for k in sorted_list1:
  with open(output_file_name, "a") as f:print("'",k,"',", sep='', file = f)
  print("'",k,"',", sep='')
with open(output_file_name, "a") as f:print("]", file = f)
print('---------------------------------------------------------------------------------')





---------------------------------------------------------------------------------
Generating Explainer
---------------------------------------------------------------------------------


Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.



                        col_name  feature_importance_vals
120                      flag_SF                 2.488795
34          dst_host_serror_rate                 2.476363
35      dst_host_srv_serror_rate                 2.391401
25                 same_srv_rate                 2.241930
65                  service_http                 2.193499
23                   rerror_rate                 2.107904
22               srv_serror_rate                 2.098623
39             Protocol_type_tcp                 2.079923
8                      logged_in                 2.024267
30        dst_host_same_srv_rate                 2.013137
24               srv_rerror_rate                 1.959203
116                      flag_S0                 1.898265
37      dst_host_srv_rerror_rate                 1.780975
28                dst_host_count                 1.768924
90               service_private                 1.761357
29            dst_host_srv_count                 1.729229
27           